In [20]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
import warnings
#env
import os
from dotenv import load_dotenv
warnings.filterwarnings('ignore')


## Extraccion

In [21]:
load_dotenv()  # Carga las variables del .env
db_url = os.getenv("DATABASE_URL")  # Obtiene la URL de la base de datos desde el .env
engine = create_engine(db_url)  # Crea el motor de conexión
connection = engine.connect()  # Abre la conexión

Tenemos que realizar 5 dimensiones
1. Categoria
2. Clientes
3. Producto
4. Proveedor
5. Tiempo

y una tabla de hechos

In [22]:
#Extraccion de las tablas de la base de datos
df_categoria = pd.read_sql_query("SELECT * FROM categories", engine)
df_clientes = pd.read_sql_query("SELECT * FROM customers", engine)
df_productos = pd.read_sql_query("SELECT * FROM products", engine)
df_proveedores = pd.read_sql_query("SELECT * FROM suppliers", engine)
df_ordenes = pd.read_sql_query("SELECT * FROM orders", engine)

Recordar que Mientras que los hechos responden a preguntas como ¿cuánto?

las dimensiones responden a preguntas como ¿quién?, ¿qué?, ¿dónde?, ¿cuándo?, ¿cómo?.

Tambien que NUNCA se tocan los datos crudos, debe de hacerse una copia la cual se debe de modificar

Dependiendo de la situacion evitar los datos nulos o duplicados

# Dimension Categorias

In [23]:
#tabla de categorias contiene los siguientes campos
# category_name, description , picture ,category_id (PK)
#Solo quiero extraer category_name, description y category_id
dim_categoria = df_categoria[['category_id', 'category_name', 'description']].copy()
dim_categoria

,category_id,category_name,description
0,1,Beverages,"Soft drinks, coffees, teas, beers, and ales"
1,2,Condiments,"Sweet and savory sauces, relishes, spreads, an..."
2,3,Confections,"Desserts, candies, and sweet breads"
3,4,Dairy Products,Cheeses
4,5,Grains/Cereals,"Breads, crackers, pasta, and cereal"
5,6,Meat/Poultry,Prepared meats
6,7,Produce,Dried fruit and bean curd
7,8,Seafood,Seaweed and fish


# Dimension Clientes

In [24]:
#tabla de clientes contiene los siguientes campos
#company_name, contact_name, contact_title, address, city, region, postal_code, country, phone, fax, customer_id (PK)
#solo quiero extraer customer_id, company_name,contact_name, city, region
dim_clientes = df_clientes[['customer_id', 'company_name', 'contact_name', 'city', 'region']].copy()
dim_clientes

,customer_id,company_name,contact_name,city,region
0,ALFKI,Alfreds Futterkiste,Maria Anders,Berlin,None
1,ANATR,Ana Trujillo Emparedados y helados,Ana Trujillo,México D.F.,None
2,ANTON,Antonio Moreno Taquería,Antonio Moreno,México D.F.,None
3,AROUT,Around the Horn,Thomas Hardy,London,None
4,BERGS,Berglunds snabbköp,Christina Berglund,Luleå,None
...,...,...,...,...,...
86,WARTH,Wartian Herkku,Pirkko Koskitalo,Oulu,None
87,WELLI,Wellington Importadora,Paula Parente,Resende,SP
88,WHITC,White Clover Markets,Karl Jablonski,Seattle,WA
89,WILMK,Wilman Kala,Matti Karttunen,Helsinki,None


# Dimension Producto

In [25]:
#tabla de productos contiene los siguientes campos
#product_name,supplier_id,quantity_per_unit,unit_price,units_in_stock,units_on_order,reorder_level,discontinued,product_id (PK),category_id (FK)
#solo quiero extrare product_id, product_name, unit_price, units_in_stock,discontinued
dim_productos = df_productos[['product_id', 'product_name', 'unit_price', 'units_in_stock', 'discontinued']].copy()
dim_productos

,product_id,product_name,unit_price,units_in_stock,discontinued
0,1,Chai,18.00,39,1
1,2,Chang,19.00,17,1
2,3,Aniseed Syrup,10.00,13,0
3,4,Chef Anton's Cajun Seasoning,22.00,53,0
4,5,Chef Anton's Gumbo Mix,21.35,0,1
...,...,...,...,...,...
72,73,Röd Kaviar,15.00,101,0
73,74,Longlife Tofu,10.00,4,0
74,75,Rhönbräu Klosterbier,7.75,125,0
75,76,Lakkalikööri,18.00,57,0


# Dimension Proveedor

In [ ]:
#tabla de Proveedores contiene los siguientes campos
#company_name, contact_name, contact_title, address, city, region, postal_code, country, phone, fax,homepage,supplier_id (PK)
#solo quiero extraer supplier_id, company_name, contact_name, city, region
dim_proveedores = df_proveedores[['supplier_id', 'company_name', 'contact_name', 'city', 'region']].copy()
dim_proveedores

,supplier_id,company_name,contact_name,city,region
0,1,Exotic Liquids,Charlotte Cooper,London,None
1,2,New Orleans Cajun Delights,Shelley Burke,New Orleans,LA
2,3,Grandma Kelly's Homestead,Regina Murphy,Ann Arbor,MI
3,4,Tokyo Traders,Yoshi Nagase,Tokyo,None
4,5,Cooperativa de Quesos 'Las Cabras',Antonio del Valle Saavedra,Oviedo,Asturias
5,6,Mayumi's,Mayumi Ohno,Osaka,None
6,7,"Pavlova, Ltd.",Ian Devling,Melbourne,Victoria
7,8,"Specialty Biscuits, Ltd.",Peter Wilson,Manchester,None
8,9,PB Knäckebröd AB,Lars Peterson,Göteborg,None
9,10,Refrescos Americanas LTDA,Carlos Diaz,Sao Paulo,None


# Dimension Tiempo

In [27]:
# la dimension tiempo no esta en si en la base de datos

In [ ]:
# Creando la dimension tiempo
dim_tiempo = df_ordenes[['order_date']].copy()

# Convertir la columna order_date a datetime
dim_tiempo['order_date'] = pd.to_datetime(dim_tiempo['order_date'], format='%Y-%m-%d')

# Extraer los componentes de fecha
dim_tiempo['mes'] = dim_tiempo['order_date'].dt.month
dim_tiempo['dia'] = dim_tiempo['order_date'].dt.day
dim_tiempo['año'] = dim_tiempo['order_date'].dt.year
dim_tiempo['nombre_mes'] = dim_tiempo['order_date'].dt.strftime('%B')
dim_tiempo['nombre_dia'] = dim_tiempo['order_date'].dt.strftime('%A')
dim_tiempo['trimestre'] = dim_tiempo['order_date'].dt.quarter

# Agregar más campos útiles para análisis
dim_tiempo['semana_año'] = dim_tiempo['order_date'].dt.isocalendar().week
dim_tiempo['dia_año'] = dim_tiempo['order_date'].dt.dayofyear
dim_tiempo['es_fin_semana'] = dim_tiempo['order_date'].dt.weekday >= 5

# Crear una clave única para la fecha (date_id)
dim_tiempo['date_id'] = dim_tiempo['order_date'].dt.strftime('%Y%m%d').astype(int)

# Eliminar duplicados para tener fechas únicas
dim_tiempo = dim_tiempo.drop_duplicates(subset=['order_date']).reset_index(drop=True)

# Reordenar las columnas
dim_tiempo = dim_tiempo[['date_id', 'order_date', 'año', 'mes', 'dia', 'trimestre', 
                         'nombre_mes', 'nombre_dia', 'semana_año', 'dia_año', 'es_fin_semana']]

dim_tiempo